In [30]:
# Importamos las librerias necesarias
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


## Carga de datos
En la práctica 4 ya realicé la limpieza de datos, por lo que en esta práctica cargaremos el dataset limpio. En la [práctica 4](https://github.com/Luisruimor/practica4_IA) hice dos posibles tratamientos de los valores nulos de la columna "age":

1. Eliminar las filas con valores nulos
2. Imputar los valores mediante el método MICE

En esta práctica vamos a trabajar con el dataset en el que eliminé las columnas con valores nulos, ya que en el método MICE se utiliza la variable que queremos predecir como parte del método de imputación.

In [31]:
# Cargamos el dataset y mostramos las primeras filas
db=pd.read_csv("train_clean_0.csv")
db.head()

## Definición de variable (recordatorio)
- Survived (si sobrevive el pasajero) = 0: Sobrevive, 1: No Sobrevive
- Pclass (la clase en la que se encontraba el pasajero ) = 1: 1a clase, 2: 2a clase, 3: 3a clase
- Sex (indica el sexo)= 0: masculino, 1: femenino
- Age (edad)
- SibSp (indica el número de hermanos, hermanastros, esposo/a que se encontraban a bordo)
- Parch (número total de padres e hijos a bordo)
- Fare (indica la tarifa/precio del ticket)
- Embarked (Indica desde donde zarpo el pasajero) = 0:Southampton ,1:Cherbourg, 2:Queenstown

In [32]:
db.describe()

## Preprocesamiento del Test Dataset

In [33]:
# Cargamos el dataset de test y mostramos las primeras filas
ExerciseTest=pd.read_csv("test.csv")
db_test = ExerciseTest.copy()
db_test.head()

In [34]:
# Eliminamos las columnas PassengerId, Ticket, Unnamed y Cabin
db_test.drop(['PassengerId','Ticket','Unnamed: 0','Cabin'],axis=1,inplace=True)

# Mapeamos las columnas Embarked y Sex
db_test["Embarked"] = db_test["Embarked"].map({"S": 0, "C": 1, "Q": 2})
db_test["Sex"] = db_test["Sex"].map({"male": 0, "female": 1})

In [35]:
# Calcular % de datos nulos por columna
for col in db_test.columns:
    print(col,":",db_test[col].isnull().sum()/db_test.shape[0]*100,"%")

In [36]:
# Eliminamos la columna Name, que no nos aporta información relevante
db.drop('Name',axis=1,inplace=True)
db_test.drop('Name',axis=1,inplace=True)

### Procesamiento de los datos nulos
#### Procesamiento de los datos nulos de la columna "Fare"

In [37]:
# Mostrar los valores nulos de la columna Fare
db_test[db_test['Fare'].isnull()]

En el caso de la columna Fare solo hay un elemento nulo por lo que decidí investigar para ver si podía completar la información a mano. En este caso Storey, Mr. Thomas era parte de la tripulación por lo que el Fare es 0.
https://www.encyclopedia-titanica.org/titanic-victim/thomas-storey.html


In [38]:
# Rellenamos el valor nulo con 0
db_test['Fare'].fillna(0,inplace=True)

#### Procesamiento de los datos nulos de la columna "Age"

In [39]:
# Calcular % de datos nulos por columna
for col in db_test.columns:
    print(col,":",db_test[col].isnull().sum()/db_test.shape[0]*100,"%")

## KNN
El método KNN clasifica datos en función de su similitud con los datos de entrenamiento existentes. La "k" en KNN especifica el nuñemero de vecinos más cercanos que se toman en cuenta para hacer la predicción.
### Preparamos datos

In [40]:
X = db.drop('Survived',axis=1) # Todas las columnas menos Survived, que es la que queremos predecir
Y = db['Survived'] # La columna que queremos predecir

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Seleccionamos el valor de k
Analizo como se comporta el modelo con diferentes valores de k, y los represento en una gráfica para ver cual es el valor de k que mejor resultado nos da.

In [41]:
# Probamos con diferentes valores de k
k_range = range(1, 20)
scores = []
for k in k_range:
      knn = KNeighborsClassifier(n_neighbors = k)
      knn.fit(X_train, Y_train)
      scores.append(knn.score(X_test, Y_test))

# Los representamos en una gráfica
plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20])

### Entrenamos el modelo con el valor de k que mejor resultado nos ha dado
Como se ve representado en la gráfica, el valor más alto de precisión del test set se obtiene con k=11, por lo que entreno el modelo con este valor de k.

In [42]:
n_neighbors = 11

# Creamos el modelo
knn = KNeighborsClassifier(n_neighbors)
# Lo entrenamos
knn.fit(X_train, Y_train)

# Imprimimos la precisión del modelo
print('Precisión del modelo en el set de entrenamiento: {:.2f}'
      .format(knn.score(X_train, Y_train)))
print('Precisión del modelo en el set de prueba: {:.2f}'
      .format(knn.score(X_test, Y_test)))

In [43]:
predictSurvive = knn.predict(db_test)

output = pd.DataFrame({'PassengerId': ExerciseTest.PassengerId, 'Survived': predictSurvive })
output.to_csv('test_kaggle_KNN.csv', index=False)
print("Your submission was successfully saved!")

## Naive Bayes

In [ ]:
X = db.drop('Survived',axis=1) # Todas las columnas menos Survived, que es la que queremos predecir
Y = db['Survived'] # La columna que queremos predecir

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Crear una instancia del clasificador Naive Bayes (GaussianNB)
clf = MultinomialNB()

# Ajustar el clasificador a los datos de entrenamiento
clf.fit(X_train, y_train)

In [ ]:
# Realizar predicciones en los datos de prueba
predictSurvive = clf.predict(db_test)
output = pd.DataFrame({'PassengerId': ExerciseTest.PassengerId, 'Survived': predictSurvive })
output.to_csv('test_kaggle_NB.csv', index=False)